In [ ]:
#stat成功率，从10到500.sampling number，间隔为10.实验50次，除去失败的，只看成功了的，计算成功了的的平均值

In [54]:
load('multi-dim-hssp.sage')
load("hssp.sage")
load("extendhssp.sage")
def generate_vector(sampling_number, data, batch_size, random_sampling = False, random_generation = False, rate = 0.5, sample_figure_length = 10):
    data = np.load(data)
    if random_generation == False:
        D=matrix(ZZ,data)
        e=vector(ones_matrix(1,batch_size))
        n = batch_size
        m0 = sampling_number
        freqD=(n+1)*[0]

        for x in e*D: 
            #print(x)
            freqD[x]+=1

        freqU=[binomial(n,i)/2.**n for i in range(n+1)]

        c=dict()
        for x in D.T: 
            h=x*e
            if h in c.keys(): c[h]+=[x]
            else: c[h]=[x]
        sampleSize=[int(m0*f) for f in freqU]
        
        if random_sampling == False:
            
            L = []
    # 假设 L 是一个列表，n 和 sampleSize 已经定义
            for i in range(n + 1):
        # 先检查键 i 是否存在于字典 c 中
                if i in c:
            # 如果存在，再检查 c[i] 的长度是否大于等于 sampleSize[i]
                    if len(c[i]) >= sampleSize[i]:
                # 如果是，正常采样
                        L += random.sample(c[i], k=sampleSize[i])
                    else:
                # 如果不是，取所有元素
                        L += c[i]
        else:
            L = []
            all_elements = []
            for key, value in c.items():
                all_elements.extend(value)
    # 然后，从这个列表中随机选取100个元素，前提是列表长度至少为100
            L += random.sample(all_elements, sampling_number)
        while True:
            random.shuffle(L)
            newX=matrix(ZZ,L)
            if newX.rank()==batch_size: break 
        
        freqnewX=(n+1)*[0]
        for x in newX*e: freqnewX[x]+=1
        G1=bar_chart(freqD, legend_label='frequency hw input')
        G2=bar_chart(freqnewX, legend_label='frequency hw output')
        G3=bar_chart(freqU, legend_label='frequency hw uniform')
        #for g in [G1,G2]: g.show()
        
        
    else:
        np.random.seed(0)

        size = (sampling_number, batch_size)
        p = [rate , 1 - rate]  # 0和1出现的概率
        #print(p)
    # 使用numpy的random.choice来生成矩阵
        random_matrix = np.random.choice([0, 1], size=size, p=p)

        
        while True:
            np.random.shuffle(random_matrix)
            newX=matrix(ZZ,random_matrix)
            if newX.rank()==batch_size: break 
        
        
    m=(newX.dimensions()[0])//batch_size*batch_size
        
    l = sample_figure_length
        
    x0,A,dummyX,dummyB=genParams_mat(m=m,l=batch_size)

    Y=newX[:m]   
   
    Y = matrix(ZZ,Y)
    print(Y.dimensions())
    return Y, x0


def matrixed_image(image,batch_size):
    alp = np.load(image)
    alp = (alp + 1.0) / 2.0

    # 将像素值缩放到[0,255]的范围
    alp = (alp * 255.0).astype(np.uint8)

    # 转置维度，以便图像数据变为(batch_size, height, width, channels)
    alp = np.transpose(alp, (0, 2, 3, 1))
    alp = alp.reshape(batch_size, -1)

    # alp=np.load("/Users/jichanglong/Desktop/hssp_All/quantize/alpha10_3072.npy")
    # print(alp.shape)
    Aifull = matrix(ZZ, alp)
    
    return Aifull

def process_image(Aifull, sample_figure_length):
    
    l = sample_figure_length
    someA=random.sample(Aifull.columns(),l)
    Ai=matrix(ZZ,someA).T
    
    return Ai

import numpy as np
from sklearn.decomposition import FastICA
import math
from time import time

def ICA(X, B=1, n=16, kappa=-1, exn=0):
    t = time()
    X = np.asarray(X)
    ncomp, sample = X.shape
    #print(X.shape)
    ica = FastICA(n_components=ncomp, fun='cube')
    S_ = ica.fit_transform(X.T)
    A_ = ica.mixing_
    # assert np.allclose(X.T, np.dot(S_, A_.T) + ica.mean_)

    # we want to remove the mean
    #print(A_)
    #print(np.linalg.inv(A_.T).shape)
    s0 = np.dot(ica.mean_, np.linalg.inv(A_.T))
    S_ += s0

    # assert np.allclose(X, np.dot(A_,S_.T))  # now the mean is 0

    # we want that the components of S_ are positive
    # if the average of a column of S_ is negative, we take the opposite
    Sm = np.sign(np.average(S_, axis=0))
    S_ *= Sm
    A_ *= Sm

    # assert np.allclose(X, np.dot(A_,S_.T))

    # we want that the components of S_ are 0 or 1
    # If all the components of S_ are 0 or 1, the standard deviation will be 1/2
    # If the components are uniformly distributed between 0 and B, the standard deviation is
    # sqrt(B(B+2)/12)
    # So we divide the components of S_ by std/sqrt(B(B+2)/12), where std is computed column by column
    if kappa == -1 and exn == 0:
        exn = math.sqrt(B * (B + 2) / 12.)
    elif exn == 0:
        exn = math.sqrt((2 * B + 1) * (B + 1) * kappa / (n * 6.) - (kappa * (B + 1) / (2 * n)) ** 2)  # exn=math.sqrt((2*B+1)*(B+1)*kappa/(n*6.))

    st = S_.std(axis=0) / exn
    S_ /= st
    A_ *= st

    # print " ICA.py %.2f" %(time()-t),
    # assert np.allclose(X, np.dot(A_,S_.T))
    np.save('./ICA/A_.npy',A_)
    
    np.save('./ICA/S.npy',S_.T)

    #return A_, S_.T
    return



import numpy as np
import random
import matplotlib.pyplot as plt
import json

def image_reconstruction(MBi, matrixB, batch_size):
    
    res_alpha = np.linalg.lstsq(MBi.T, matrixB, rcond=None)[0]

    # print(res_alpha.shape)
    # print('Reconstructed input using NS attack:')
    num_col = batch_size
    if batch_size >20:
        num_col = batch_size//4
    num_row = int(batch_size / num_col)
    #num_samples = num_row * num_col
    shape = (32, 32, 3)
    fig, axes = plt.subplots(num_row, num_col, figsize=(1. * num_col, 1. * num_row))
    for i, ax in enumerate(axes.flat):
        ax.set_axis_off()
        ax.imshow(res_alpha[i].reshape(shape).astype(np.uint8), cmap='gray')
#     plt.savefig('./figure/cifar_cat_40/cifar10-NS.pdf')  # 保存为 PDF
#     plt.savefig('./figure/cifar_cat_40/cifar10-NS.png', dpi=600)  # 保存为 PNG，指定分辨率为300 dpi


In [114]:
#special_version

def All_attack_special(batch_size, modulus, processed_image, generated_vector, sample_figure_length, round_number):
    n = batch_size
    x0 = modulus
    Ai = processed_image
    Y = generated_vector
    m = Y.dimensions()[0]
    count =True
    if m not in [70,100,130,160,190,220,250,280,310,340,370,400,430,460, 500]:
        count = False
    l = sample_figure_length
    ss = 0
    while True and count == True:
        rows_list = list(Y.rows())
        random.shuffle(rows_list)
        Y = matrix(rows_list)
        Bi = Y * Ai % x0
        s = Matrix(Integers(x0), Bi[:l, :l]).determinant()
        ss = ss +1
        if s != 0 or ss == 100:
            print(ss)
            break
    
    # 数据结构
    attack_data = {
        'm': round_number,
        'Attack Type': '',
        'Error Type': 0,
        'Time': None,
        'NFound': None,
        'Coef': None
    }
    

#     attack_data['Attack Type'] = 'NS Attack'
#     start_time = time()
#     try:
#         if ss<100 and count == True:
#             kei, tt1, tt10, tt1O = Step1_original(n, -1, x0, Ai, Y, Bi, m)
#             MBi, beta = Step2_BKZ(kei, 1, n, m, x0, Y, 1, 1, -1)
#             NFound = batch_size
#             Coef = batch_size
#         else: MBi  = 0
#         if MBi == 0:
#             attack_data['Error Type'] = 1
#         elif MBi == 1:
#             attack_data['Error Type'] = 2
#         else:
#             attack_data.update({'Time': time() - start_time, 'NFound': NFound, 'Coef': Coef})
#     except TimeoutException:
#         attack_data['Error Type'] = 'Exception: Timeout'
#     except Exception as e:
#         attack_data['Error Type'] = 'Exception: ' + str(e)
#     results.append(attack_data.copy())

#     #Multi攻击
#     attack_data['Attack Type'] = 'Multi Attack'
#     start_time = time()
#     try:
#         #MO, tt1, tt10, tt1O = Step1_Mat(n, x0, Ai, Y, Bi, m)
#         MO, tt1, tt10, tt1O = Step1(n, -1, x0, Ai, Y, Bi, m, BKZ=True)
#         tei, tef, tt2, nrafound, MB, NFound, Coef = eigen_mat(n, -1, MO, x0, Ai, Y, Bi, m)
#         if MB == 0:
#             attack_data['Error Type'] = 3
#             attack_data.update({'NFound': 0, 'Coef':0})
#         else:
#             attack_data.update({'Time': time() - start_time, 'NFound': NFound, 'Coef': Coef})
#     except Exception as e:
#         attack_data['Error Type'] = 6 
#         attack_data.update({'NFound': 0, 'Coef':0})
#     results.append(attack_data.copy())

#     # Statistical攻击
    attack_data['Attack Type'] = 'Statistical Attack'
    start_time = time()
    try:
        MO, tt1, tt10, tt1O = Step1(n, -1, x0, Ai, Y, Bi, m, BKZ=True)
        if MO == -1:
            return 
        MOn, MO = statistical_1(MO, n, m, x0, Y, Ai, Bi, -1)
        ICA(MOn)  
        tica, tt2, nrafound, MB, NFound, Coef = statistical_2_mat(MOn, MO, n, m, x0, Y, Ai, Bi, -1)
        if MB == 0:
            attack_data['Error Type'] = 4
        elif not np.all(np.floor(MB) == MB): 
            attack_data['Error Type'] = 5
        else:
            attack_data.update({'Time': time() - start_time, 'NFound': NFound, 'Coef': Coef})
    except Exception as e:
        attack_data['Error Type'] = 'Exception: ' + str(e)
    results.append(attack_data.copy())


In [133]:
#special_version

def All_attack_special_70_80(batch_size, modulus, processed_image, generated_vector, sample_figure_length, round_number):
    n = batch_size
    x0 = modulus
    Ai = processed_image
    Y = generated_vector
    m = Y.dimensions()[0]
    count =True
    if m not in [70,80]:
        count = False
    l = sample_figure_length
    ss = 0
    while True and count == True:
        rows_list = list(Y.rows())
        random.shuffle(rows_list)
        Y = matrix(rows_list)
        Bi = Y * Ai % x0
        s = Matrix(Integers(x0), Bi[:l, :l]).determinant()
        ss = ss +1
        if s != 0 or ss == 100:
            print(ss)
            break
    
    # 数据结构
    attack_data = {
        'm': round_number,
        'Attack Type': '',
        'Error Type': 0,
        'Time': None,
        'NFound': None,
        'Coef': None
    }
    

#     attack_data['Attack Type'] = 'NS Attack'
#     start_time = time()
#     try:
#         if ss<100 and count == True:
#             kei, tt1, tt10, tt1O = Step1_original(n, -1, x0, Ai, Y, Bi, m)
#             MBi, beta = Step2_BKZ(kei, 1, n, m, x0, Y, 1, 1, -1)
#             NFound = batch_size
#             Coef = batch_size
#         else: MBi  = 0
#         if MBi == 0:
#             attack_data['Error Type'] = 1
#         elif MBi == 1:
#             attack_data['Error Type'] = 2
#         else:
#             attack_data.update({'Time': time() - start_time, 'NFound': NFound, 'Coef': Coef})
#     except TimeoutException:
#         attack_data['Error Type'] = 'Exception: Timeout'
#     except Exception as e:
#         attack_data['Error Type'] = 'Exception: ' + str(e)
#     results.append(attack_data.copy())

#     #Multi攻击
#     attack_data['Attack Type'] = 'Multi Attack'
#     start_time = time()
#     try:
#         #MO, tt1, tt10, tt1O = Step1_Mat(n, x0, Ai, Y, Bi, m)
#         MO, tt1, tt10, tt1O = Step1(n, -1, x0, Ai, Y, Bi, m, BKZ=True)
#         tei, tef, tt2, nrafound, MB, NFound, Coef = eigen_mat(n, -1, MO, x0, Ai, Y, Bi, m)
#         if MB == 0:
#             attack_data['Error Type'] = 3
#             attack_data.update({'NFound': 0, 'Coef':0})
#         else:
#             attack_data.update({'Time': time() - start_time, 'NFound': NFound, 'Coef': Coef})
#     except Exception as e:
#         attack_data['Error Type'] = 6 
#         attack_data.update({'NFound': 0, 'Coef':0})
#     results.append(attack_data.copy())

#     # Statistical攻击
    attack_data['Attack Type'] = 'Statistical Attack'
    start_time = time()
    try:
        MO, tt1, tt10, tt1O = Step1(n, -1, x0, Ai, Y, Bi, m, BKZ=True)
        if MO == -1:
            return 
        MOn, MO = statistical_1(MO, n, m, x0, Y, Ai, Bi, -1)
        ICA(MOn)  
        tica, tt2, nrafound, MB, NFound, Coef = statistical_2_mat(MOn, MO, n, m, x0, Y, Ai, Bi, -1)
        if MB == 0:
            attack_data['Error Type'] = 4
        elif not np.all(np.floor(MB) == MB): 
            attack_data['Error Type'] = 5
        else:
            attack_data.update({'Time': time() - start_time, 'NFound': NFound, 'Coef': Coef})
    except Exception as e:
        attack_data['Error Type'] = 'Exception: ' + str(e)
    results.append(attack_data.copy())


In [78]:
load("hssp.sage")
load('multi-dim-hssp.sage')
load("extendhssp.sage")
load("statistical.sage")
conf_path = './utils/conf.json'

    # 读取配置文件
with open(conf_path, 'r') as f:
    conf = json.load(f)
    
size =10


relu_data = f"./expdata/expdata_batchsize_{size}/gradients_fc1_relu_binary_epoch_1_batch_0.npy"

original_image = f"./expdata/expdata_batchsize_{size}/images_epoch_1_batch_0.npy"

sampling_number=10

batch_size=conf["batch_size"]


batch_size=size
print(batch_size)
random_sampling = True

random_generation = False

rate = 0.5

sample_figure_length = 10

test_rounds = 50


10


In [88]:
import signal
from time import time

# 定义一个超时异常类
class TimeoutException(Exception):
    pass

# 超时处理函数
def timeout_handler(signum, frame):
    raise TimeoutException

# 设置信号处理器
signal.signal(signal.SIGALRM, timeout_handler)


<Handlers.SIG_DFL: 0>

In [135]:
#50个model的攻击stat

# 假设在代码开始之前已经有了一个初始的sampling_number值
initial_sampling_number = 10

# 外部循环从1到50
for run_index in range(1, 1000):
    print(f"执行序号 {run_index}")
    results = []
    # 每次外部循环开始，重置sampling_number到初始值
    sampling_number = initial_sampling_number
    bach = (run_index+2)%50
    if bach == 0:
        bach =1
    relu_data = f"/Users/jichanglong/Desktop/hssp_new/expdata/initialization_purchase_cifar/purchase/initial_activation.npy"

    original_image = f"/Users/jichanglong/Desktop/hssp_new/expdata/initialization_purchase_cifar/purchase/images_epoch_10_batch_0.npy"
    
    m_list = [70,80]
    # 内部循环，重复16次
    for i in range(2):
        print(f"第{i+1}轮")
        sampling_number =m_list[i]
        print(sampling_number)
        
        if sampling_number not in [70,80]:
            continue
        # 生成向量和处理图像的逻辑...
        generated_vector, modulus = generate_vector(sampling_number, relu_data, batch_size, random_sampling, random_generation, rate, sample_figure_length)

        full_image = matrixed_image(original_image, batch_size)
        processed_image = process_image(full_image, sample_figure_length)
        All_attack_special(batch_size, modulus, processed_image, generated_vector, sample_figure_length, i + 1)

    # 写入结果到文件，文件名中包含当前的run_index
    print(results)
    with open(f'/Users/jichanglong/Desktop/hssp_new/comparisonData/True_fl/purchase_ini/docu_stat_{run_index}.txt', 'w') as f:
        for result in results:
            f.write(f"Round: {result['m']}, Attack Type: {result['Attack Type']}, Error: {result['Error Type']}, Time: {result['Time']}, NFound: {result['NFound']}, Coef: {result['Coef']}\n")


执行序号 1
第1轮
70
(70, 10)


ValueError: axes don't match array

In [132]:
#50个model的攻击stat

# 假设在代码开始之前已经有了一个初始的sampling_number值
initial_sampling_number = 10

# 外部循环从1到50
for run_index in range(139, 150):
    print(f"执行序号 {run_index}")
    results = []
    # 每次外部循环开始，重置sampling_number到初始值
    sampling_number = initial_sampling_number
    bach = (run_index+2)%50
    if bach == 0:
        bach =1
    relu_data = f"/Users/jichanglong/Desktop/hssp_new/expdata/true_fl/cifar10/truefl/gradients_fc1_relu_binary_epoch_{bach}_batch_0.npy"

    original_image = f"/Users/jichanglong/Desktop/hssp_new/expdata/true_fl/cifar10/truefl/images_epoch_{bach}_batch_0.npy"

    # 内部循环，重复16次
    for i in range(49):
        print(f"第{i+1}轮")
        sampling_number += 10
        print(sampling_number)
        

        # 生成向量和处理图像的逻辑...
        generated_vector, modulus = generate_vector(sampling_number, relu_data, batch_size, random_sampling, random_generation, rate, sample_figure_length)

        full_image = matrixed_image(original_image, batch_size)
        processed_image = process_image(full_image, sample_figure_length)
        All_attack_special(batch_size, modulus, processed_image, generated_vector, sample_figure_length, i + 1)

    # 写入结果到文件，文件名中包含当前的run_index
    print(results)
    with open(f'/Users/jichanglong/Desktop/hssp_new/comparisonData/True_fl/cifar/docu_stat_{run_index}.txt', 'w') as f:
        for result in results:
            f.write(f"Round: {result['m']}, Attack Type: {result['Attack Type']}, Error: {result['Error Type']}, Time: {result['Time']}, NFound: {result['NFound']}, Coef: {result['Coef']}\n")


执行序号 139
第1轮
20
(20, 10)
第2轮
30
(30, 10)
第3轮
40
(40, 10)
第4轮
50
(50, 10)
第5轮
60
(60, 10)
第6轮
70
(70, 10)
8
Building M 
Step 1
  LLL:0.0
  Sred1:0.0
  Sred2:0.0
  #ortho vecs= 60 out of 60
  Kernel LLL: 0.0
  Total Step 1: 0.0
Step 2-ICA:  roundA
 time LLL= 0.0006009999924572185 mathNbits= 1
statistical_2_mat here
time Ica_A:  0.00023999999393709004
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  1  0  0  0  0  0  0  0]
[ 0  0  1  0  0  0  0  0  1  0]
[ 0  0  0  1  0  1  0  0  0 -1]
[ 0  0  0  0  1  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0 -1  0]
第7轮
80
(80, 10)
第8轮
90
(90, 10)
第9轮
100
(100, 10)
2
Building M 
Step 1
  LLL:0.0
  Sred1:0.0
  Sred2:0.0
  #ortho vecs= 90 out of 90
  Kernel LLL: 0.0
  Total Step 1: 0.0
Step 2-ICA:  roundA
 time LLL= 0.0008119999984046444 mathNbits= 2
statistical_2_mat here
time Ica_A:  0.0008600000001024455
[ 1  0  0  0  0  0  0  0  0  0]
[ 

  Sred1:0.1
  Sred2:0.2
  #ortho vecs= 450 out of 450
  Kernel LLL: 0.0
  Total Step 1: 0.4
Step 2-ICA:  roundA
 time LLL= 0.003548000007867813 mathNbits= 2
statistical_2_mat here
time Ica_A:  0.00047300000733230263
[ 0  0  0  0  0  0  1  0  0  0]
[ 0  0  0  0  0  0  0  0  0  1]
[ 0  1  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0 -1  0]
[ 0  0  0  0  0  0  0  0  1  0]
[ 0  0  0  1  0  0  0  0  0  0]
[ 1  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  1  0  0]
[ 0  0  1  0  0  0  0  0 -1  0]
[ 0  0  0  0  0  1  0 -1  0  0]
第46轮
470
(470, 10)
第47轮
480
(480, 10)
第48轮
490
(490, 10)
第49轮
500
(500, 10)
55
Building M 
Step 1
  LLL:0.0
  Sred1:0.1
  Sred2:0.2
  #ortho vecs= 490 out of 490
  Kernel LLL: 0.0
  Total Step 1: 0.4
Step 2-ICA:  roundA
 time LLL= 0.0038120000099297613 mathNbits= 2
statistical_2_mat here
time Ica_A:  0.0007110000005923212
[ 0  0  0  0  1  0  0  0  0  0]
[ 1  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  1  0  0  0]
[ 0  0  0  0  0  0  0 -1  0  0]
[ 0  0  0  0

7
Building M 
Step 1
  LLL:0.0
  Sred1:0.0
  Sred2:0.0
  #ortho vecs= 60 out of 60
  Kernel LLL: 0.0
  Total Step 1: 0.0
Step 2-ICA:  roundA
 time LLL= 0.0006379999977070838 mathNbits= 1
statistical_2_mat here
time Ica_A:  0.0002789999853121117
[ 1  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  1  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0 -1  0  0]
[ 0  0  0  0  1  0  0  0  0 -1]
[ 0  0 -1  0  0  0  0  0  0  0]
[ 0  1  0  0  0  0  0  0  0  0]
[-1  0  0  1  0  0  0  0  0  0]
[ 0  0  0  0  0  1 -1  0 -1  0]
第7轮
80
(80, 10)
第8轮
90
(90, 10)
第9轮
100
(100, 10)
1
Building M 
Step 1
  LLL:0.0
  Sred1:0.0
  Sred2:0.0
  #ortho vecs= 90 out of 90
  Kernel LLL: 0.0
  Total Step 1: 0.0
Step 2-ICA:  roundA
 time LLL= 0.0008349999989150092 mathNbits= 1
statistical_2_mat here
time Ica_A:  0.0004810000100405887
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0

23
Building M 
Step 1
  LLL:0.0
  Sred1:0.1
  Sred2:0.2
  #ortho vecs= 490 out of 490
  Kernel LLL: 0.0
  Total Step 1: 0.4
Step 2-ICA:  roundA
 time LLL= 0.0038139999960549176 mathNbits= 1
statistical_2_mat here
time Ica_A:  0.0005530000053113326
[ 0  1  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  1  0  0  0  0  0  0]
[ 1  0  0  0  0  0  0  0  0  0]
[ 0  0  1  0  0  1  0  0  0 -1]
[ 0  0  0  0  0  0  0  0  1  0]
[ 0  0  0  0  1  0  0  0  0  0]
[ 0  0  0  0  0  0 -1  0  0  0]
[ 0  0  0 -1  0  0  0 -1  0  0]
[{'m': 1, 'Attack Type': 'Statistical Attack', 'Error Type': "Exception: cannot access local variable 'Bi' where it is not associated with a value", 'Time': None, 'NFound': None, 'Coef': None}, {'m': 2, 'Attack Type': 'Statistical Attack', 'Error Type': "Exception: cannot access local variable 'Bi' where it is not associated with a value", 'Time': None, 'NFound': None, 'Coef': None}, {'m': 3, 'Attack Type': 'Statistical Attack',

第6轮
70
(70, 10)
61
Building M 
Step 1
  LLL:0.0
  Sred1:0.0
  Sred2:0.0
  #ortho vecs= 60 out of 60
  Kernel LLL: 0.0
  Total Step 1: 0.0
Step 2-ICA:  roundA
 time LLL= 0.0008010000019567087 mathNbits= 1
statistical_2_mat here
time Ica_A:  0.0007389999955194071
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0 -1  0  0]
[ 0  0  0  0  1  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0 -1  0  0  0  0  0  0]
[ 0  0  0 -1 -1  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  1  0  0  1  0  0  0  0]
第7轮
80
(80, 10)
第8轮
90
(90, 10)
第9轮
100
(100, 10)
44
Building M 
Step 1
  LLL:0.0
  Sred1:0.0
  Sred2:0.0
  #ortho vecs= 90 out of 90
  Kernel LLL: 0.0
  Total Step 1: 0.0
Step 2-ICA:  roundA
 time LLL= 0.000933999996050261 mathNbits= 1
statistical_2_mat here
time Ica_A:  0.00023900001542642713
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  

第49轮
500
(500, 10)
7
Building M 
Step 1
  LLL:0.0
  Sred1:0.1
  Sred2:0.2
  #ortho vecs= 490 out of 490
  Kernel LLL: 0.0
  Total Step 1: 0.4
Step 2-ICA:  roundA
 time LLL= 0.0037089999968884513 mathNbits= 1
statistical_2_mat here
time Ica_A:  0.000590000010561198
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0 -1]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0 -1  0  0  0  0  0]
[ 0  0  0  0  0  1  0  0  0  0]
[ 0  0  0  1  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  1  0  0]
[ 0  0  0  0  0  0  0  0  1  0]
[{'m': 1, 'Attack Type': 'Statistical Attack', 'Error Type': "Exception: cannot access local variable 'Bi' where it is not associated with a value", 'Time': None, 'NFound': None, 'Coef': None}, {'m': 2, 'Attack Type': 'Statistical Attack', 'Error Type': "Exception: cannot access local variable 'Bi' where it is not associated with a value", 'Time': None, 'NFound': None, 'Coef': None}, {'m': 3, 'Attack Type': 'Sta

第4轮
50
(50, 10)
第5轮
60
(60, 10)
第6轮
70
(70, 10)
1
Building M 
Step 1
  LLL:0.0
  Sred1:0.0
  Sred2:0.0
  #ortho vecs= 60 out of 60
  Kernel LLL: 0.0
  Total Step 1: 0.0
Step 2-ICA:  roundA
 time LLL= 0.0006719999946653843 mathNbits= 1
statistical_2_mat here
time Ica_A:  0.0006980000034673139
[ 0  0  0  0  0  0  0  0  0  0]
[ 1  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0 -1  0  0  0  0  0  0]
[ 0  0 -1  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0 -1  0  0  0  0  0  0  0  0]
[ 0 -1  0 -1  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0 -1]
第7轮
80
(80, 10)
第8轮
90
(90, 10)
第9轮
100
(100, 10)
1
Building M 
Step 1
  LLL:0.0
  Sred1:0.0
  Sred2:0.0
  #ortho vecs= 90 out of 90
  Kernel LLL: 0.0
  Total Step 1: 0.0
Step 2-ICA:  roundA
 time LLL= 0.0008400000078836456 mathNbits= 1
statistical_2_mat here
time Ica_A:  0.0005470000178320333
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0 

第49轮
500
(500, 10)
18
Building M 
Step 1
  LLL:0.0
  Sred1:0.1
  Sred2:0.2
  #ortho vecs= 490 out of 490
  Kernel LLL: 0.0
  Total Step 1: 0.4
Step 2-ICA:  roundA
 time LLL= 0.0037980000051902607 mathNbits= 1
statistical_2_mat here
time Ica_A:  0.00021299999207258224
[ 0  0  0  0  1  0  0  0  0  0]
[ 0  0  0  0  0  0  0  1  0  0]
[ 0  0  1  0  0  0  0  0  0  0]
[ 0  0  1  0  0  0  0 -1  0  0]
[ 1  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  1  0]
[ 0  0  0  0  0  1  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  1]
[ 0  1 -1  0  0  0  0  0  0  0]
[ 0  0  0 -1  0  0 -1  1  0  0]
[{'m': 1, 'Attack Type': 'Statistical Attack', 'Error Type': "Exception: cannot access local variable 'Bi' where it is not associated with a value", 'Time': None, 'NFound': None, 'Coef': None}, {'m': 2, 'Attack Type': 'Statistical Attack', 'Error Type': "Exception: cannot access local variable 'Bi' where it is not associated with a value", 'Time': None, 'NFound': None, 'Coef': None}, {'m': 3, 'Attack Type': '

KeyboardInterrupt: 

In [25]:
relu_data = f"/Users/jichanglong/Desktop/hssp_new/expdata/true_fl/cifar10/truefl/gradients_fc1_relu_binary_epoch_1_batch_0.npy"

In [27]:
r = np.load(relu_data)
print(r.shape)

(10, 500)
